#**Análisis de datos**

##A - ETL (Extracción, Transformación y Carga)

###1. Extracción

In [197]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/AD_agro/Estimaciones.csv'
with open(ruta_archivo, 'rb') as f:
    result = chardet.detect(f.read())

try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=';')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')

Mounted at /content/drive
Extración de datos exitosa


###2. Transformación

In [198]:
df.head()

,ID Provincia,Provincia,ID Departamento,Departamento,Id Cultivo,Cultivo,ID Campaña,Campana,Sup. Sembrada (Ha),Sup. Cosechada (Ha),Producción (Tn),Rendimiento (Kg/Ha)
0,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,1,1969/70,3,3,10,3333
1,6,BUENOS AIRES,854,25 DE MAYO,1,Ajo,2,1970/71,1,1,3,3000
2,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,1,1969/70,15,15,82,5467
3,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,2,1970/71,10,10,55,5500
4,6,BUENOS AIRES,14,ADOLFO GONZALES CHAVES,1,Ajo,3,1971/72,8,8,44,5500


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160327 entries, 0 to 160326
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID Provincia         160327 non-null  int64 
 1   Provincia            160327 non-null  object
 2   ID Departamento      160327 non-null  int64 
 3   Departamento         160327 non-null  object
 4   Id Cultivo           160327 non-null  int64 
 5   Cultivo              160327 non-null  object
 6   ID Campaña           160327 non-null  int64 
 7   Campana              160327 non-null  object
 8   Sup. Sembrada (Ha)   160327 non-null  int64 
 9   Sup. Cosechada (Ha)  160327 non-null  int64 
 10  Producción (Tn)      160327 non-null  int64 
 11  Rendimiento (Kg/Ha)  160327 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 14.7+ MB


In [200]:
# Renombrar columnas para facilitar el acceso
df.rename(columns ={
    'ID Provincia': 'id_provincia',
    'Provincia': 'provincia',
    'ID Departamento': 'id_departamento',
    'Departamento': 'departamento',
    'Id Cultivo': 'id_cultivo',
    'Cultivo': 'cultivo',
    'ID Campaña': 'id_campana',
    'Campana': 'campana',
    'Sup. Sembrada (Ha)': 'superficie_sembrada',
    'Sup. Cosechada (Ha)': 'superficie_cosechada',
    'Producción (Tn)': 'produccion',
    'Rendimiento (Kg/Ha)': 'rendimiento',
}, inplace=True)

#Eliminar columnas innecesarias:
df.drop(columns=['id_provincia', 'id_departamento', 'id_cultivo'], inplace=True)

# Crear una columna de SUPERFICIE PERDIDA en hectáreas, en base a la diferencia entre los valores de las columnas superficie_sembrada y superficie_cosechada.
df['superficie_perdida'] = (df['superficie_sembrada'] - df['superficie_cosechada'])

# Limpiar y Corrección de los datos de la columna 'id_campana':
df['id_campana'] = df['id_campana'].replace({57: 55, 58: 56})

#Cambiar la posición de la columna sup_perdida para una mejor organización y visualización de las columnas
col= df.pop('superficie_perdida')
df.insert(7, 'superficie_perdida', col)

print("Transformación de datos exitosa")

Transformación de datos exitosa


###3. Carga

In [201]:
# definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/AD_agro/estimaciones_nuevas.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Google dRIVE
df.to_csv(ruta_salida, index=False, sep=',')

print(f'Archivo CSV guardado en: {ruta_salida}')

Archivo CSV guardado en: /content/drive/MyDrive/AD_agro/estimaciones_nuevas.csv


#B. EDA (Análisis Exploratorio de Datos)

In [202]:
#Analisis general de los datos por columnas (Nombre de columna, conteo y tipo de datos)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160327 entries, 0 to 160326
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   provincia             160327 non-null  object
 1   departamento          160327 non-null  object
 2   cultivo               160327 non-null  object
 3   id_campana            160327 non-null  int64 
 4   campana               160327 non-null  object
 5   superficie_sembrada   160327 non-null  int64 
 6   superficie_cosechada  160327 non-null  int64 
 7   superficie_perdida    160327 non-null  int64 
 8   produccion            160327 non-null  int64 
 9   rendimiento           160327 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 12.2+ MB


In [203]:
#Estadisitica descriptiva del dataframe:
print(df.describe().apply(lambda s: s.apply(lambda x: f'{x:,.2f}')))

       id_campana superficie_sembrada superficie_cosechada superficie_perdida  \
count  160,327.00          160,327.00           160,327.00         160,327.00   
mean        25.45           12,566.32            11,022.08           1,544.25   
std         17.05           36,275.18            34,787.78           6,241.65   
min          1.00                0.00                 0.00          -7,130.00   
25%         10.00              200.00                73.00               0.00   
50%         22.00            1,500.00               800.00              20.00   
75%         41.00            9,000.00             6,250.00             660.00   
max         56.00          894,613.00           883,613.00         620,000.00   

         produccion rendimiento  
count    160,327.00  160,327.00  
mean      34,398.28    3,458.76  
std      130,124.32    5,478.63  
min            0.00        0.00  
25%          196.00    1,000.00  
50%        1,814.00    2,000.00  
75%       14,782.00    3,500.00 

In [204]:
#Información sobre las columnas categóricas:

#Provincia
provincias = df['provincia'].unique()
print("PROVINCIAS:")
print(provincias)

#Departamento:
departamento = df['departamento'].unique()
print(" \n DEPARTAMENTOS:")
print(departamento)

#Cultivos:
cultivos = df['cultivo'].unique()
print(" \n CULTIVOS:")
print(cultivos)

#Campaña
campana = len(df['campana'].unique())
print(" \n CAMPAÑAS:")
print(campana)


PROVINCIAS:
['BUENOS AIRES' 'CATAMARCA' 'CHACO' 'CHUBUT' 'CORDOBA' 'CORRIENTES'
 'ENTRE RIOS' 'FORMOSA' 'JUJUY' 'LA PAMPA' 'LA RIOJA' 'MENDOZA' 'MISIONES'
 'NEUQUEN' 'RIO NEGRO' 'SALTA' 'SAN JUAN' 'SAN LUIS' 'SANTA CRUZ'
 'SANTA FE' 'SANTIAGO DEL ESTERO' 'TUCUMAN' 'TIERRA DEL FUEGO']
 
 DEPARTAMENTOS:
['25 DE MAYO' 'ADOLFO GONZALES CHAVES' 'ALBERTI' 'ALMIRANTE BROWN'
 'ARRECIFES' 'AYACUCHO' 'AZUL' 'BAHIA BLANCA' 'BALCARCE' 'BARADERO'
 'BENITO JUAREZ' 'BERAZATEGUI' 'BERISSO' 'BOLIVAR' 'BRANDSEN' 'CAMPANA'
 'CANUELAS' 'CAPITAN SARMIENTO' 'CARMEN DE ARECO' 'CASTELLI' 'CHACABUCO'
 'CHASCOMUS' 'CHIVILCOY' 'COLON' 'CORONEL DE MARINA L ROSALES'
 'CORONEL DORREGO' 'CORONEL PRINGLES' 'CORONEL SUAREZ' 'DOLORES' 'ESCOBAR'
 'ESTEBAN ECHEVERRIA' 'EXALTACION DE LA CRUZ' 'FLORENCIO VARELA'
 'GENERAL ALVARADO' 'GENERAL ARENALES' 'GENERAL BELGRANO'
 'GENERAL JUAN MADARIAGA' 'GENERAL LA MADRID' 'GENERAL PUEYRREDON'
 'GENERAL RODRIGUEZ' 'JUNIN' 'LA MATANZA' 'LA PLATA' 'LAPRIDA'
 'LAS FLORES' 'LOBERIA' 'L

In [205]:
#campana = df[df['id_campana'] > 52]
campana = df['id_campana'].reset_index().sort_values(by='id_campana', ascending=False)
print (campana)

         index  id_campana
73243    73243          56
14860    14860          56
110364  110364          56
142872  142872          56
14807    14807          56
...        ...         ...
66541    66541           1
134667  134667           1
22500    22500           1
40710    40710           1
40734    40734           1

[160327 rows x 2 columns]


In [206]:
#1.Cual fue la superficie cosechada de maiz por provincia en la campaña 2023/2024?

df_Sup_maiz = df[(df['cultivo'] == 'Maíz') & (df['id_campana'] == 55)]
sup_cosecha_maiz_provincia_2024 = df_Sup_maiz.groupby('provincia')['superficie_cosechada'].sum().reset_index().sort_values(by='superficie_cosechada', ascending=False)

print(sup_cosecha_maiz_provincia_2024)

              provincia  superficie_cosechada
3               CORDOBA               2826211
0          BUENOS AIRES               2418516
12             SANTA FE                851091
8              LA PAMPA                559270
13  SANTIAGO DEL ESTERO                512275
5            ENTRE RIOS                447900
11             SAN LUIS                446150
10                SALTA                371492
2                 CHACO                135900
14              TUCUMAN                 81080
9              MISIONES                 30659
6               FORMOSA                 18612
1             CATAMARCA                 14890
7                 JUJUY                 11666
4            CORRIENTES                 11000


In [207]:
#2.Qué provincia tiene el máximo valor de produccion de soja históricamente?

df_produccion_soja = df[df['cultivo'] == 'Soja total']
produccion_promedio_soja = df_produccion_soja.groupby('provincia')['produccion'].sum().reset_index().sort_values(by='produccion', ascending=False)

print(produccion_promedio_soja.head())


              provincia  produccion
0          BUENOS AIRES   377502319
3               CORDOBA   368942411
13             SANTA FE   328863757
5            ENTRE RIOS    74207443
14  SANTIAGO DEL ESTERO    57652369


In [208]:
#3.Cúal es el rendimiento máximo por cultivo y por provincia?
table_max_rendimiento_cultivo_provincia = pd.pivot_table(
    df,
    values='rendimiento',
    index='provincia',
    columns='cultivo',
    aggfunc='max')
print(table_max_rendimiento_cultivo_provincia)

cultivo                  Ajo  Algodón  Alpiste   Arroz  Arveja   Avena  \
provincia                                                                
BUENOS AIRES          9900.0      NaN   4700.0     NaN  4000.0  4000.0   
CATAMARCA             4500.0   4500.0      NaN     NaN     NaN  1000.0   
CHACO                 5000.0   3000.0      NaN  7000.0     NaN     0.0   
CHUBUT                6233.0      NaN      0.0     NaN     NaN  1790.0   
CORDOBA              15400.0   4000.0   1800.0     NaN  3199.0  4100.0   
CORRIENTES            3167.0   2800.0      NaN  9000.0     NaN     0.0   
ENTRE RIOS            4375.0   2200.0   2000.0  8750.0  2800.0  3200.0   
FORMOSA               5500.0   3500.0      NaN  6800.0     NaN     0.0   
JUJUY                 7000.0   2700.0      NaN     NaN     NaN  1400.0   
LA PAMPA              7000.0      NaN   1200.0     NaN     NaN  3000.0   
LA RIOJA              7100.0   3481.0      NaN     NaN     NaN     0.0   
MENDOZA              11023.0    667.0 